## Project 2       Jungang (Gordon) Chen            UIN: 5280097929

## 1)  Load the text dataset and text preprocessing

In [404]:
import pickle
import numpy as np
import tensorflow as tf

In [405]:
input_sequence = pickle.load(open('DS_5_train_input', 'rb'))
output_sequence = pickle.load(open('DS_5_train_output', 'rb'))
print(type(input_sequence))

<class 'list'>


In [407]:
text_pairs = []
for line in range(len(input_sequence)):
    inputish  = input_sequence[line]
    outputish = "[start] " + output_sequence[line] + " [end]"
    text_pairs.append((inputish, outputish))

import random
print(random.choice(text_pairs))

('a f b e a j a h a g b e b d a e c f c g a j b d a j b d a h c e b d c g a g a k c e b e a d b e b g b e b f ', '[start] b e b e b d a g e f c f a e h c g a h g i j b d b d c e b d c g a h ed ee ef c e b e a k eh ei b e b g a d ek el a g ej em a j m eg fd b e a j l fe ff b f a j k fg fh a f d fi  [end]')


In [408]:
import random
random.shuffle(text_pairs)
print(len(text_pairs))
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

5000


In [409]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import string
import re

strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

src_vocab_size = 50
tgt_vocab_size = 50
sequence_length = 100

source_vectorization = layers.TextVectorization(
    max_tokens=src_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=tgt_vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length+1, 
    standardize=custom_standardization,
)

train_inputish_texts = [pair[0] for pair in train_pairs]
train_outputish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_inputish_texts)
target_vectorization.adapt(train_outputish_texts)
print(len(source_vectorization.get_vocabulary()))

print(len(target_vectorization.get_vocabulary()))

13
38


In [410]:
batch_size = 16
def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

print(train_ds)

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, 100), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 100), dtype=tf.int64, name=None))>


In [411]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (16, 100)
inputs['spanish'].shape: (16, 100)
targets.shape: (16, 100)


## 2) Build a transformer model with a stack of 2 encoder and 3 decoder

In [412]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [413]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [414]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [415]:
embed_dim = 64
dense_dim = 512
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, src_vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, tgt_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(tgt_vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [416]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint("artificial_text_translation.keras",
                                    save_best_only=True)
]
transformer.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)

Epoch 1/100
219/219 [==============================] - 17s 43ms/step - loss: 1.3770 - accuracy: 0.3043 - val_loss: 0.9866 - val_accuracy: 0.4040
Epoch 2/100
219/219 [==============================] - 8s 37ms/step - loss: 0.9653 - accuracy: 0.4255 - val_loss: 0.8357 - val_accuracy: 0.4911
Epoch 3/100
219/219 [==============================] - 8s 37ms/step - loss: 0.8214 - accuracy: 0.4957 - val_loss: 0.7000 - val_accuracy: 0.5534
Epoch 4/100
219/219 [==============================] - 8s 38ms/step - loss: 0.7360 - accuracy: 0.5395 - val_loss: 0.6550 - val_accuracy: 0.5818
Epoch 5/100
219/219 [==============================] - 8s 38ms/step - loss: 0.6840 - accuracy: 0.5691 - val_loss: 0.6505 - val_accuracy: 0.5867
Epoch 6/100
219/219 [==============================] - 8s 37ms/step - loss: 0.6292 - accuracy: 0.5999 - val_loss: 0.6134 - val_accuracy: 0.6048
Epoch 7/100
219/219 [==============================] - 8s 37ms/step - loss: 0.5788 - accuracy: 0.6312 - val_loss: 0.6065 - val_accuracy

## 3) Model Accuracy by randomly selecting 20 true/prediction testing sentences and compare them token by token

In [422]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 100

def decode_sequence(input_sentence, model):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = model(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

def transform_format(true_output_sequence, pred_sequence):
    true_output_sequence = true_output_sequence.split()
    pred_sequence  = pred_sequence.split()
    true_output_sequence.remove('[start]')
    true_output_sequence.remove('[end]')
    pred_sequence.remove('[start]')
    pred_sequence.remove('[end]')
    return true_output_sequence, pred_sequence


test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]

count_total = 0
count_match =0 
for i in np.random.randint(len(test_eng_texts),size=20):
    input_sequence = test_eng_texts[i]
    true_output_sequence = test_spa_texts[i]    
    pred_sequence  = decode_sequence(input_sequence,transformer)
    
    true_output_sequence, pred_sequence=transform_format(true_output_sequence, pred_sequence)
    count_total+=len(true_output_sequence)
    for i, token in enumerate(pred_sequence):
        if i<len(true_output_sequence):
            if token == true_output_sequence[i]:
            count_match+=1
        else:
            break
  # print(count_match)
  # print(len(true_output_sequence))
  # print(count_match/len(true_output_sequence))

print(count_match/count_total)

0.7979966611018364


## 4) Reload the model and test again

In [423]:
# At loading time, register the custom objects with a `custom_object_scope`:
load_model = keras.models.load_model("artificial_text_translation.keras", custom_objects={
'TransformerEncoder': TransformerEncoder,
'TransformerDecoder': TransformerDecoder,
'PositionalEmbedding': PositionalEmbedding
})

test_eng_texts = [pair[0] for pair in test_pairs]
test_spa_texts = [pair[1] for pair in test_pairs]

count_total = 0
count_match = 0 
for i in np.random.randint(len(test_eng_texts),size=20):
    input_sequence = test_eng_texts[i]
    true_output_sequence = test_spa_texts[i]    
    pred_sequence  = decode_sequence(input_sequence,load_model)

    true_output_sequence, pred_sequence=transform_format(true_output_sequence, pred_sequence)
    count_total+=len(true_output_sequence)
    for i, token in enumerate(pred_sequence):
        if i<len(true_output_sequence):
            if token == true_output_sequence[i]:
                count_match+=1
        else:
            break

print(count_match/count_total)

0.7939581719597212
